In [37]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import bs4

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline

## Задание: необходимо построить классификатор для определения тональности отзыва для https://www.kaggle.com/c/morecomplicatedsentiment/overview

In [38]:
# посмотрим на тестовые данные
with open('test.csv', 'r', encoding='utf-8') as f:
    reviews = f.read()
    
# выделим текст между тегами review и избавимся от '\n' в тексте
texts = bs4.BeautifulSoup(reviews, 'lxml').find_all('review')
texts = [item.text.replace("\n", " ").strip() for item in texts]
# преобразуем в pandas.DataFrame
test = pd.DataFrame(texts, columns=['data'])
test.head()

,data
0,"Ужасно слабый аккумулятор, это основной минус ..."
1,ценанадежность-неубиваемостьдолго держит батар...
2,"подробнее в комментариях К сожалению, факт пол..."
3,я любительница громкой музыки. Тише телефона у...
4,"Дата выпуска - 2011 г, емкость - 1430 mAh, тех..."


Судя по тестовой выборке, мы тестируем отзывы на телефоны, поэтому я собрал тренировочную выборку с сайта https://otziv-otziv.ru/katalog/mobilnye-telefony/. Для парсинга запускаем скрипт parse_otziv.py, который создает следующая ячека:

In [39]:
%%writefile parse_otziv.py
import requests
import bs4
import pandas as pd
from multiprocessing import Pool
from functools import reduce

def parse_page(url):
    parsed_data = []
    try:
        # парсим заданную страницу
        req = requests.get(url)
        print(str(url), req)
        parser = bs4.BeautifulSoup(req.text, 'lxml')
        # ищем ссылки на отдельные модели
        divs_models = parser.findAll('div', attrs={'class':'content'})
        refs = [div.find('a', href=True)['href'] for div in divs_models[:-1]]
    
        # парсим каждую страницу для отдельно взятой модели телефона
        for ref in refs:
            req2 = requests.get('https://otziv-otziv.ru' + ref)
            parser2 = bs4.BeautifulSoup(req2.text, 'lxml')
            # поиск тексты отзывов
            divs_feedback = parser2.findAll('div', attrs={'class':'container-reviews collapsible collapsed'})
            
            # для каждого отзыва ищем комментарий и рейтинг
            for feedback in divs_feedback:
                # из всего отзыва берем только текст после слова 'Комментарий:', если он есть
                comment_ind = feedback.text.find('Комментарий:')
                if comment_ind == -1:
                    continue          # комментария нет
                else:
                    comment = feedback.text[comment_ind+12:].replace('\n', ' ').strip()
                    rating = int(feedback.find('div', attrs={'class':'stars-container'}).attrs['title'][-1])
                    parsed_data.append((comment, rating))
    except requests.exceptions.ConnectionError:
        print('ConnectionError to URL ' + url)
        parsed_data.append(('отстойный телефон', 1))
        
    return parsed_data
                    
                    
if __name__ == '__main__':
    p = Pool(10)
    url_list = ['https://otziv-otziv.ru/katalog/mobilnye-telefony/?page=' + str(n) for n in range(1, 709)]
    map_results = list(p.map(parse_page, url_list))
    reduce_results = reduce(lambda x,y: x + y, map_results)
    train_raw = pd.DataFrame(reduce_results, columns=['data', 'rating'])
    train_raw[['data','rating']].to_csv('train_raw.csv', index = False)

Writing parse_otziv.py


In [40]:
# на выходе получаем файл 'train_raw.csv'. Посмотрим на данные:
train_raw = pd.read_csv('train_raw.csv')
train_raw

,data,rating
0,Телефон норм.,4
1,"Вобщем телефон отличный и долговечный , недост...",5
2,у меня был sony ericsson xperia x8 и теперь no...,5
3,"Пользуюсь третий год, купил на замену нокии 62...",3
4,пока альтернатив как простому телефону с хорош...,5
...,...,...
188296,Смартфон для требовательных игр наверное слабо...,4
188297,Заказывал на известном китайском сайте. Особых...,4
188298,"Заказал из Китая за 7200р, т.к. у нас этой мод...",4
188299,"Не посоветовал бы никому. Есть второй телефон,...",3


In [41]:
# будем считать, что рейтинг 4-5 это положительный отзыв, а 1-3 отрицательный. Добавим колонку 'label':
train_raw['label'] = np.where(train_raw['rating']>=4, 1, 0)
# заодно отбросим NA занчения, если такие есть в таблице
train_raw.dropna(inplace=True)
train_raw

,data,rating,label
0,Телефон норм.,4,1
1,"Вобщем телефон отличный и долговечный , недост...",5,1
2,у меня был sony ericsson xperia x8 и теперь no...,5,1
3,"Пользуюсь третий год, купил на замену нокии 62...",3,0
4,пока альтернатив как простому телефону с хорош...,5,1
...,...,...,...
188296,Смартфон для требовательных игр наверное слабо...,4,1
188297,Заказывал на известном китайском сайте. Особых...,4,1
188298,"Заказал из Китая за 7200р, т.к. у нас этой мод...",4,1
188299,"Не посоветовал бы никому. Есть второй телефон,...",3,0


In [42]:
# посмотрим на соотношение классов
train_raw['label'].value_counts()

1    133245
0     55050
Name: label, dtype: int64

In [43]:
# положительных отзывов значительно больше, чем отрицательных.
# произведем балансировку классов
from sklearn.utils import resample, shuffle

train_majority = train_raw[train_raw.label==1]
train_minority = train_raw[train_raw.label==0]
# т.к. данных много, то просто возьмем 55050 случайных положительных значений
train_majority_downsampled = resample(train_majority, 
                                      replace=False,
                                      n_samples=55050)
 
# объединим положительные и отрицательные значения и перемешаем
train = shuffle(pd.concat([train_majority_downsampled, train_minority]))
# проверим баланс
train.label.value_counts()

1    55050
0    55050
Name: label, dtype: int64

In [44]:
train

,data,rating,label
13658,Спустя 2 года пользования поменял свое мнение ...,3,0
19389,сдал его обратно только из-за дефекта в режиме...,4,1
32938,Кожаный флип хоть и защищает тлефон от царапин...,1,0
29615,Нет возможности установить разблокировку отпеч...,3,0
39142,"Никаких наворотов нет: интернета, фотокамеры и...",5,1
...,...,...,...
11575,не могу сказать что тел совсем плохой ну если ...,3,0
65848,Купила неделю назад. Пару раз самопроизвольно ...,3,0
163320,Прибрел коммуникатор Keneksi Norma. Очень непл...,4,1
48118,Больше пока ничего не изучила.Полчаса открывал...,3,0


In [45]:
# запишем 'train' в файл на будущее.
# будем использовать колонки 'data' и 'label'
train[['data','label']].to_csv('train.csv', index=False)

In [55]:
train = pd.read_csv('train.csv')
print(f'Количество отзывов в обучающей выборке: {len(train)}')
portion_1 = sum(train.label) / len(train.label)
print(f'Доля позитивных отзывов: {portion_1}')
print(f'Количесто отзывов в тестовой выборке: {len(test)}')

Количество отзывов в обучающей выборке: 110100
Доля позитивных отзывов: 0.5
Количесто отзывов в тестовой выборке: 100


### Определим наиболее перспективную модель

In [48]:
%%time
clf_accuracies = {}
for vect in [CountVectorizer(), TfidfVectorizer()]:
    for clf in [LogisticRegression(solver='liblinear'), LinearSVC()]:
        pipe = Pipeline([("vect", vect), ("clf", clf)])
        clf_accuracies[(vect,clf)] = cross_val_score(pipe, train.data, train.label, n_jobs=-1).mean()
        
clf_accuracies

Wall time: 2min 47s


{(CountVectorizer(),
  LogisticRegression(solver='liblinear')): 0.8516893732970028,
 (CountVectorizer(), LinearSVC()): 0.8498546775658491,
 (TfidfVectorizer(),
  LogisticRegression(solver='liblinear')): 0.8350499545867394,
 (TfidfVectorizer(), LinearSVC()): 0.8556039963669392}

### Лучшее качество показало сочетание TfidfVectorizer и LinearSVC, подберем для данной модели параметры

In [49]:
# Создадим Pipeline будем использовать TfidfVectorizer и LinearSVC
pipe = Pipeline([('vect', TfidfVectorizer()), 
                 ('clf', LinearSVC())])

# Для подбора параметров используем GridSearchCV
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2), (1, 3)),                      # униграмы, биграмы или триграмы
    'clf__C': (1, 10, 100),                                             # параметр регуляризации
    'clf__max_iter': (10, 25, 50)                                       # максимальное количество итераций
}

model = GridSearchCV(pipe, parameters, n_jobs=-1)

In [50]:
%%time
# поиск наилучших параметров для модели
model.fit(train.data, train.label)
# точность модели с наилучшими параметрами
print(f'Точность лучшей модели: {model.best_score_:.3f}')

Точность лучшей модели: 0.898
Wall time: 21min 48s


In [51]:
# выбранные для модели параметры
model.best_estimator_

Pipeline(steps=[('vect', TfidfVectorizer(ngram_range=(1, 3))),
                ('clf', LinearSVC(C=1, max_iter=10))])

In [52]:
%%time
pipe_final = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 3))),
                       ('clf', LinearSVC(C=1, max_iter=10))])

pipe_final.fit(train.data, train.label)

# используем полученную модель для предсказания
test['prediction'] = pipe_final.predict(test.data)

Wall time: 49.2 s


In [53]:
test['Id'] = test.index
# преобразуем единицы и нули в значения 'pos' и 'neg' необходимые для финального решения
test['y'] = np.where(test['prediction']==0, 'neg', 'pos')
test

,data,prediction,Id,y
0,"Ужасно слабый аккумулятор, это основной минус ...",0,0,neg
1,ценанадежность-неубиваемостьдолго держит батар...,1,1,pos
2,"подробнее в комментариях К сожалению, факт пол...",0,2,neg
3,я любительница громкой музыки. Тише телефона у...,0,3,neg
4,"Дата выпуска - 2011 г, емкость - 1430 mAh, тех...",1,4,pos
...,...,...,...,...
95,"Нет передней камеры, внутренняя память очень м...",0,95,neg
96,"Звук при прослушивание музыки хороший,не глючи...",1,96,pos
97,Очень маленькая память забита вшитыми и соверш...,0,97,neg
98,"Удобный корпус,стандартное меню нокиа,камера д...",1,98,pos


In [54]:
# Запишем в файл решение для загрузки на Kaggle
test[['Id','y']].to_csv('FINAL_submission.csv', index=False)